<a href="https://colab.research.google.com/github/ArunVignesh75/Machine-Learning/blob/main/NLP_Entity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Finding Entities from tweets and Its Sentiment Analysis

#Dataset Information:

      **You are given a JSON file (tweets.json) that contains tweets (sentences) along with the name of the author.**

Objective 1: Get the most frequent entities from the tweets.

Objective 2: Find out the sentiment/polarity of each author towards each of the entities.

1.Import Packages 

2.Read Data 

3.Understand and Preprocessing the Data

3.1- Converting Data to CSV format

3.2- Transposing Row & Columns

4- Using spacy to Identify Entities

5- Finding the Frequency of Entities

6- Saving Objective 1 as CSV file

7- Using Sentiment Intensity Analysis Model

8- Finding Polarity values of Entities

9- Saving Sentiment Analysis to CSV file


Importing Necessary Packages

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

import nltk
nltk.download('stopwords')
nltk.download('brown')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer


In [ ]:

from nltk import word_tokenize, pos_tag, ne_chunk
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
import pandas as pd
import numpy as np
import json
import spacy
import re
from nltk.corpus import stopwords 
from nltk.util import ngrams 
from collections import Counter

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading Small English model "en_core_web_sm" from the spaCy library

In [ ]:
nlp = spacy.load("en_core_web_sm")

Loading the Dataset

In [ ]:
tweets = "tweets.json"
df = pd.read_json(tweets,
   convert_dates=True)

Transposing Rows & Columns

In [ ]:
df = df.T

In [ ]:
df

,tweet_author,tweet_text
2013-07-18 09:39:46.071961602,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...
2013-07-17 03:40:32.173842437,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...
2013-07-15 15:41:16.553048065,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...
2013-07-12 19:19:42.367813635,Toby Eyre,#acalabrutinib is a valuable option in pts int...
2013-07-04 12:40:34.334232586,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...
...,...,...
1987-06-19 12:17:53.643945985,Joy is a Lifestyle,Hanging out with Friends! :) #FF #CLL #Happine...
1987-06-19 12:06:26.675290112,𝓒𝓻𝓲𝔃𝔃𝔂 𝓟𝓮𝓻𝓻𝔂🌹,Hanging out with Friends! :) #FF #CLL #Happine...
1987-06-17 23:05:41.186953217,IQWiG,Zusatznutzen von #Idelalisib ist weder für #CL...
1987-06-17 15:18:00.525635584,Medibooks,#Hematología PTK2 EXPRESSION AND IMMUNOCHEMOTH...


In [ ]:
df['id'] = df.index

In [ ]:
df

,tweet_author,tweet_text,id
2013-07-18 09:39:46.071961602,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...,2013-07-18 09:39:46.071961602
2013-07-17 03:40:32.173842437,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...,2013-07-17 03:40:32.173842437
2013-07-15 15:41:16.553048065,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...,2013-07-15 15:41:16.553048065
2013-07-12 19:19:42.367813635,Toby Eyre,#acalabrutinib is a valuable option in pts int...,2013-07-12 19:19:42.367813635
2013-07-04 12:40:34.334232586,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...,2013-07-04 12:40:34.334232586
...,...,...,...
1987-06-19 12:17:53.643945985,Joy is a Lifestyle,Hanging out with Friends! :) #FF #CLL #Happine...,1987-06-19 12:17:53.643945985
1987-06-19 12:06:26.675290112,𝓒𝓻𝓲𝔃𝔃𝔂 𝓟𝓮𝓻𝓻𝔂🌹,Hanging out with Friends! :) #FF #CLL #Happine...,1987-06-19 12:06:26.675290112
1987-06-17 23:05:41.186953217,IQWiG,Zusatznutzen von #Idelalisib ist weder für #CL...,1987-06-17 23:05:41.186953217
1987-06-17 15:18:00.525635584,Medibooks,#Hematología PTK2 EXPRESSION AND IMMUNOCHEMOTH...,1987-06-17 15:18:00.525635584


Drop the Index Values

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43347 entries, 0 to 43346
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   tweet_author  43347 non-null  object        
 1   tweet_text    43347 non-null  object        
 2   id            43347 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 1016.1+ KB


In [ ]:
df.columns

Index(['tweet_author', 'tweet_text', 'id'], dtype='object')

In [ ]:
df.shape

(43347, 3)

In [ ]:
df

,tweet_author,tweet_text,id
0,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...,2013-07-18 09:39:46.071961602
1,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...,2013-07-17 03:40:32.173842437
2,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...,2013-07-15 15:41:16.553048065
3,Toby Eyre,#acalabrutinib is a valuable option in pts int...,2013-07-12 19:19:42.367813635
4,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...,2013-07-04 12:40:34.334232586
...,...,...,...
43342,Joy is a Lifestyle,Hanging out with Friends! :) #FF #CLL #Happine...,1987-06-19 12:17:53.643945985
43343,𝓒𝓻𝓲𝔃𝔃𝔂 𝓟𝓮𝓻𝓻𝔂🌹,Hanging out with Friends! :) #FF #CLL #Happine...,1987-06-19 12:06:26.675290112
43344,IQWiG,Zusatznutzen von #Idelalisib ist weder für #CL...,1987-06-17 23:05:41.186953217
43345,Medibooks,#Hematología PTK2 EXPRESSION AND IMMUNOCHEMOTH...,1987-06-17 15:18:00.525635584


Data Pre-processing

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt   

In [ ]:
df['tweets1'] = np.vectorize(remove_pattern)(df['tweet_text'], "#[\w]*")
df['tweets1'] = np.vectorize(remove_pattern)(df['tweets1'], "https[\w]*")
df['tweets1'] = np.vectorize(remove_pattern)(df['tweets1'], "https|RT|@[\w]*")
df['tweets1'] = df['tweets1'].str.replace("[^a-zA-Z#]", " ")
df['tweets1'] = df['tweets1'].str.lower()
stop_words = set(stopwords.words('english')) 
df['tweets1'] = df['tweets1'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

<ipython-input-18-017364dea372>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tweets1'] = df['tweets1'].str.replace("[^a-zA-Z#]", " ")


In [ ]:
df['tweets1'] = [' '.join([w for w in x.lower().split() if w not in stop_words]) 
    for x in df['tweets1'].tolist()]

In [ ]:
df

,tweet_author,tweet_text,id,tweets1
0,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...,2013-07-18 09:39:46.071961602,scientists conducted phase study acalabrutinib...
1,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...,2013-07-17 03:40:32.173842437,phase acalabrutinib venetoclax trial still rec...
2,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...,2013-07-15 15:41:16.553048065,backs lpdogra
3,Toby Eyre,#acalabrutinib is a valuable option in pts int...,2013-07-12 19:19:42.367813635,valuable option pts intolerant valuable data h...
4,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...,2013-07-04 12:40:34.334232586,nice recommended use acalabrutinib patients tr...
...,...,...,...,...
43342,Joy is a Lifestyle,Hanging out with Friends! :) #FF #CLL #Happine...,1987-06-19 12:17:53.643945985,hanging friends http pdleww bbb
43343,𝓒𝓻𝓲𝔃𝔃𝔂 𝓟𝓮𝓻𝓻𝔂🌹,Hanging out with Friends! :) #FF #CLL #Happine...,1987-06-19 12:06:26.675290112,hanging friends http gynchdnul
43344,IQWiG,Zusatznutzen von #Idelalisib ist weder für #CL...,1987-06-17 23:05:41.186953217,zusatznutzen von ist weder noch refrakt res fo...
43345,Medibooks,#Hematología PTK2 EXPRESSION AND IMMUNOCHEMOTH...,1987-06-17 15:18:00.525635584,ptk expression immunochemotherapy outcome chro...


#Entity Extraction

In [ ]:
from collections import Counter
from string import punctuation
nlp = spacy.load("en_core_web_sm")
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] 
    doc = nlp(text.lower()) 
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return result

entities = df['tweets1'].apply(get_hotwords)
df1 = entities

In [ ]:
df1

0        [scientists, phase, study, acalabrutinib, pati...
1        [phase, acalabrutinib, venetoclax, trial, recr...
2                                                [lpdogra]
3        [valuable, option, intolerant, valuable, data,...
4        [nice, recommended, use, acalabrutinib, patien...
                               ...                        
43342                                    [friends, pdleww]
43343                                 [friends, gynchdnul]
43344    [zusatznutzen, von, ist, weder, noch, refrakt,...
43345    [ptk, expression, immunochemotherapy, outcome,...
43346    [mutations, myd, pathway, subset, young, chron...
Name: tweets1, Length: 43347, dtype: object

In [ ]:
df['entity'] = df1

In [ ]:
df

,tweet_author,tweet_text,id,tweets1,entity
0,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...,2013-07-18 09:39:46.071961602,scientists conducted phase study acalabrutinib...,"[scientists, phase, study, acalabrutinib, pati..."
1,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...,2013-07-17 03:40:32.173842437,phase acalabrutinib venetoclax trial still rec...,"[phase, acalabrutinib, venetoclax, trial, recr..."
2,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...,2013-07-15 15:41:16.553048065,backs lpdogra,[lpdogra]
3,Toby Eyre,#acalabrutinib is a valuable option in pts int...,2013-07-12 19:19:42.367813635,valuable option pts intolerant valuable data h...,"[valuable, option, intolerant, valuable, data,..."
4,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...,2013-07-04 12:40:34.334232586,nice recommended use acalabrutinib patients tr...,"[nice, recommended, use, acalabrutinib, patien..."
...,...,...,...,...,...
43342,Joy is a Lifestyle,Hanging out with Friends! :) #FF #CLL #Happine...,1987-06-19 12:17:53.643945985,hanging friends http pdleww bbb,"[friends, pdleww]"
43343,𝓒𝓻𝓲𝔃𝔃𝔂 𝓟𝓮𝓻𝓻𝔂🌹,Hanging out with Friends! :) #FF #CLL #Happine...,1987-06-19 12:06:26.675290112,hanging friends http gynchdnul,"[friends, gynchdnul]"
43344,IQWiG,Zusatznutzen von #Idelalisib ist weder für #CL...,1987-06-17 23:05:41.186953217,zusatznutzen von ist weder noch refrakt res fo...,"[zusatznutzen, von, ist, weder, noch, refrakt,..."
43345,Medibooks,#Hematología PTK2 EXPRESSION AND IMMUNOCHEMOTH...,1987-06-17 15:18:00.525635584,ptk expression immunochemotherapy outcome chro...,"[ptk, expression, immunochemotherapy, outcome,..."


#Frequency of Entities

In [ ]:
from collections import Counter

entities = df["entity"].tolist()
entities = [e for sublist in entities for e in sublist if isinstance(e, str)]
frequency = dict(Counter(entities))
print(frequency)

{'scientists': 60, 'phase': 1611, 'study': 2082, 'acalabrutinib': 2794, 'patients': 8207, 'refractory': 1556, 'ibrutinib': 3132, 'intolerant': 81, 'overall': 172, 'response': 641, 'rate': 211, 'qpc': 1, 'kuzz': 1, 'venetoclax': 2077, 'trial': 2163, 'recruitment': 16, 'non': 632, 'respondent': 1, 'initial': 177, 'therapy': 2206, 'lpdogra': 1, 'valuable': 21, 'option': 326, 'data': 1538, 'decision': 68, 'early': 544, 'view': 57, 'haematologica': 28, 'kclzax': 1, 'nice': 165, 'recommended': 2, 'use': 244, 'treatment': 5467, 'chronic': 17010, 'lymphocytic': 17262, 'leukemia': 15314, 'backs': 10, 'astrazeneca': 1189, 'calquence': 2613, 'cll': 6577, 'jopnlnid': 1, 'pwy': 3, 'england': 56, 'decisions': 78, 'wales': 5, 'info': 208, 'zshh': 1, 'ofdwedlav': 1, 'chemotherapy': 338, 'free': 798, 'monotherapy': 200, 'find': 24, 'qyuft': 1, 'sxve': 1, 'nggx': 1, 'superstar': 4, 'excellent': 144, 'news': 798, 'approval': 618, 'frontline': 353, 'unsuitable': 5, 'drug': 1768, 'yesterday': 47, 'new': 56

In [ ]:
frequency_list = list(frequency.items())
frequency_df = pd.DataFrame(frequency_list, columns=['entity', 'frequency'])
frequency_df['entity'] = frequency_df['entity'].apply(lambda x: ''.join([w for w in x]))


Creating New Dataframe with Entity and Frequency

In [ ]:
frequency_df

,entity,frequency
0,scientists,60
1,phase,1611
2,study,2082
3,acalabrutinib,2794
4,patients,8207
...,...,...
63134,gynchdnul,1
63135,weder,1
63136,ptk,1
63137,rwziokjnka,1


In [ ]:
df['tweet_text'][0]

'⚕️ Scientists conducted a Phase II study of acalabrutinib in patients with relapsed/refractory #CLL who were ibrutinib-intolerant, and found an overall response rate of 73%. \nhttps://t.co/eJ6m4QpC5P https://t.co/kuZz6ZO47r'

In [ ]:
df['entity'][0]

['scientists',
 'phase',
 'study',
 'acalabrutinib',
 'patients',
 'refractory',
 'ibrutinib',
 'intolerant',
 'overall',
 'response',
 'rate',
 'qpc',
 'kuzz']

In [ ]:
frequency

{'scientists': 60,
 'phase': 1611,
 'study': 2082,
 'acalabrutinib': 2794,
 'patients': 8207,
 'refractory': 1556,
 'ibrutinib': 3132,
 'intolerant': 81,
 'overall': 172,
 'response': 641,
 'rate': 211,
 'qpc': 1,
 'kuzz': 1,
 'venetoclax': 2077,
 'trial': 2163,
 'recruitment': 16,
 'non': 632,
 'respondent': 1,
 'initial': 177,
 'therapy': 2206,
 'lpdogra': 1,
 'valuable': 21,
 'option': 326,
 'data': 1538,
 'decision': 68,
 'early': 544,
 'view': 57,
 'haematologica': 28,
 'kclzax': 1,
 'nice': 165,
 'recommended': 2,
 'use': 244,
 'treatment': 5467,
 'chronic': 17010,
 'lymphocytic': 17262,
 'leukemia': 15314,
 'backs': 10,
 'astrazeneca': 1189,
 'calquence': 2613,
 'cll': 6577,
 'jopnlnid': 1,
 'pwy': 3,
 'england': 56,
 'decisions': 78,
 'wales': 5,
 'info': 208,
 'zshh': 1,
 'ofdwedlav': 1,
 'chemotherapy': 338,
 'free': 798,
 'monotherapy': 200,
 'find': 24,
 'qyuft': 1,
 'sxve': 1,
 'nggx': 1,
 'superstar': 4,
 'excellent': 144,
 'news': 798,
 'approval': 618,
 'frontline': 353

In [ ]:
#Saving Entity and its Frequency to CSV file 

In [ ]:
frequency_df.to_csv("objective1.csv")

In [ ]:
max_length = df['tweets1'].str.len().max()
max_length

266

#Sentiment Analysis

In [ ]:
#importing Nesscessary Packages

In [ ]:
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('vader_lexicon')

stop_words = set(stopwords.words("english"))
wordnet_lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Tokenization

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

def tokenize(text):
    return word_tokenize(text)

df['tweets1_tokens'] = df['tweets1'].apply(tokenize)

In [ ]:
df.tweets1[1]

'phase acalabrutinib venetoclax trial still recruitment phase study well venetoclax acalabrutinib works mcl patients either relapsed non respondent initial therapy'

In [ ]:
df.columns

Index(['tweet_author', 'tweet_text', 'id', 'tweets1', 'entity',
       'tweets1_tokens'],
      dtype='object')

In [ ]:
df.tweets1_tokens[1]

['phase',
 'acalabrutinib',
 'venetoclax',
 'trial',
 'still',
 'recruitment',
 'phase',
 'study',
 'well',
 'venetoclax',
 'acalabrutinib',
 'works',
 'mcl',
 'patients',
 'either',
 'relapsed',
 'non',
 'respondent',
 'initial',
 'therapy']

#Initializing Sentiment Intensity Analysis Model

In [ ]:

# Initialize the SentimentIntensityAnalyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# Iterate over the rows of the 'tweets1_tokens' column and calculate the polarity values
polarity_values = []
for tokens in df['tweets1_tokens']:
    polarity = sentiment_analyzer.polarity_scores(" ".join(tokens))['compound']
    polarity_values.append(polarity)

# Create a new column in the dataframe to store the polarity values
df['polarity_values'] = polarity_values

# Map the polarity values to binary values (0 for negative, 1 for positive)
df['sentiment'] = df['polarity_values'].apply(lambda x: 1 if x >= 0 else 0)


In [ ]:
df['sentiment'] = df['sentiment'].apply(lambda x: "positive" if x==1 else "negative")


In [ ]:
df.sentiment

0        positive
1        positive
2        negative
3        positive
4        positive
           ...   
43342    positive
43343    positive
43344    positive
43345    positive
43346    positive
Name: sentiment, Length: 43347, dtype: object

Creating New Dataframe for Tweet_author, Entity and its Sentiment Values

In [ ]:
sentiment_df = pd.DataFrame(columns=['entity','author_name', 'overall_polarity'])

In [ ]:
sentiment_df['entity'] = df['entity']


In [ ]:
sentiment_df['author_name'] = df['tweet_author']

In [ ]:
sentiment_df['overall_polarity'] = df['sentiment']

In [ ]:
sentiment_df

,entity,author_name,overall_polarity
0,"[scientists, phase, study, acalabrutinib, pati...",Hematopoiesis News,positive
1,"[phase, acalabrutinib, venetoclax, trial, recr...","Michael Wang, MD",positive
2,[lpdogra],1stOncology,negative
3,"[valuable, option, intolerant, valuable, data,...",Toby Eyre,positive
4,"[nice, recommended, use, acalabrutinib, patien...",Lymphoma Hub,positive
...,...,...,...
43342,"[friends, pdleww]",Joy is a Lifestyle,positive
43343,"[friends, gynchdnul]",𝓒𝓻𝓲𝔃𝔃𝔂 𝓟𝓮𝓻𝓻𝔂🌹,positive
43344,"[zusatznutzen, von, ist, weder, noch, refrakt,...",IQWiG,positive
43345,"[ptk, expression, immunochemotherapy, outcome,...",Medibooks,positive


#Saving the Dataframe to a CSV file

In [ ]:
sentiment_df.to_csv("objective2.csv")